In [1]:
import logging

import pandas as pd

import core.config as cconfig
import data_schema.dataset_schema_utils as dsdascut
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hio as hio
import helpers.hprint as hprint
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe.universe as imvcounun

/app/core/statistics/random_samples.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-3e7567f3-5a18-428f-9cdd-7b0ff6b8381d.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.13.0' != container_version='1.12.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmampTask7396_Add_QA_for_realtime_resampled_1_min_bidask_data_collection'
  hash='2f298ee1d'
  # Last commits:
    * 2f298ee1d Sonya Nikiforova CmTask7274: Add backticks to multistring finding function (#7391) (  23 hours ago) Tue Feb 27 16:18:09 2024  (HEAD -> CmampTask7396_Add_QA_for_realtime_resampled_1_min_bidask_data_collection, origin/master, origin/HEAD, origin/CmampTask7396_Add_QA_for_realtime_resampled_1_min_bidas

In [3]:
env_var_name = "CK_DATA_RECONCILIATION_CONFIG"
config = cconfig.Config.from_env_var(env_var_name)
if config:
    config = config.to_dict()
    # bid_ask_accuracy needs to be cast to int if its defined
    config["bid_ask_accuracy"] = (
        int(config["bid_ask_accuracy"]) if config["bid_ask_accuracy"] else None
    )
    # Get config from env when running the notebook via the `run_notebook.py`
    # script, e.g., in the system reconciliation flow.
    _LOG.info("Using config from env vars")
else:
    config_dict = {
        "stage": "preprod",
        "start_timestamp": "2024-02-28T14:40:00+00:00",
        "end_timestamp": "2024-02-28T14:50:00+00:00",
        "aws_profile": "ck",
        "dataset_signature": "realtime.airflow.downloaded_200ms.postgres.bid_ask.futures.v7_3.ccxt.binance.v1_0_0",
        "bid_ask_accuracy": 1,
        "data_type": "bid_ask",
        "bid_ask_depth": 1,
        "bid_ask_frequency_sec": "60S",
    }
    config = cconfig.Config.from_dict(config_dict)
print(config)

WARNING Environment variable 'CK_DATA_RECONCILIATION_CONFIG' not defined: no config retrieved
stage: preprod
start_timestamp: 2024-02-28T14:40:00+00:00
end_timestamp: 2024-02-28T14:50:00+00:00
aws_profile: ck
dataset_signature: realtime.airflow.downloaded_200ms.postgres.bid_ask.futures.v7_3.ccxt.binance.v1_0_0
bid_ask_accuracy: 1
data_type: bid_ask
bid_ask_depth: 1
bid_ask_frequency_sec: 60S


In [4]:
dataset_schema = dsdascut.get_dataset_schema()
dsdascut.validate_dataset_signature(config["dataset_signature"], dataset_schema)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


True

In [5]:
dataset_signature_as_dict = dsdascut.parse_dataset_signature_to_args(
    config["dataset_signature"], dataset_schema
)
dataset_signature_as_dict

{'download_mode': 'realtime',
 'downloading_entity': 'airflow',
 'action_tag': 'downloaded_200ms',
 'data_format': 'postgres',
 'data_type': 'bid_ask',
 'asset_type': 'futures',
 'universe': 'v7_3',
 'vendor': 'ccxt',
 'exchange_id': 'binance',
 'version': 'v1_0_0'}

In [6]:
raw_data_client = imvcdcimrdc.RawDataReader(
    config["dataset_signature"], stage=config["stage"]
)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
INFO  Unable to fetch DB credentials from env file: 
	
################################################################################
* Failed assertion *
File '/app/im_v2/devops/env/preprod.im_db_config.env' doesn't exist
################################################################################

	Attempting AWS SecretsManager method.
INFO  Fetching secret: preprod.im_data_db
INFO  Created preprod DB connection: 
 None
INFO  Enabled connection to the `ccxt_bid_ask_futures_raw` DB table


In [7]:
data = raw_data_client.read_data(
    pd.Timestamp(config["start_timestamp"]),
    pd.Timestamp(config["end_timestamp"]),
    bid_ask_levels=[i for i in range(1, config["bid_ask_depth"] + 1)],
)

INFO  Executing query: 
	SELECT * FROM ccxt_bid_ask_futures_raw WHERE timestamp >= 1709131200000 AND timestamp <= 1709131800000 AND level IN (1) AND exchange_id = 'binance'


/app/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [8]:
data.head()

,currency_pair,exchange_id,end_download_timestamp,knowledge_timestamp,bid_size_l1,bid_price_l1,ask_size_l1,ask_price_l1
timestamp,,,,,,,,
1709131200036,CTK_USDT,binance,2024-02-28 14:40:00.340435+00:00,2024-02-28 14:40:00.362887+00:00,612.0,0.8074,178.0,0.8077
1709131200042,GMT_USDT,binance,2024-02-28 14:40:00.328794+00:00,2024-02-28 14:40:00.362887+00:00,37734.0,0.2933,30330.0,0.2934
1709131200044,NEAR_USDT,binance,2024-02-28 14:40:00.335128+00:00,2024-02-28 14:40:00.362887+00:00,2431.0,3.9320,1594.0,3.9330
1709131200045,DYDX_USDT,binance,2024-02-28 14:40:00.331477+00:00,2024-02-28 14:40:00.362887+00:00,788.5,3.5530,47.6,3.5540
1709131200055,MATIC_USDT,binance,2024-02-28 14:40:00.331131+00:00,2024-02-28 14:40:00.362887+00:00,1624.0,1.0422,2642.0,1.0423


In [9]:
# TODO(Juraj): this behavior should be encapsulated in some utility function
data_frequency = "T" if "1min" in dataset_signature_as_dict["action_tag"] else "S"
vendor_name = dataset_signature_as_dict["vendor"].upper()
mode = "download"
version = dataset_signature_as_dict["universe"].replace("_", ".")
exchange_id = dataset_signature_as_dict["exchange_id"]
universe = imvcounun.get_vendor_universe(vendor_name, mode, version=version)
universe_list = universe[exchange_id]

In [10]:
qa_check_list = [
    imvcdqqach.NaNChecks(),
    imvcdqqach.FullUniversePresentCheck(universe_list),
    imvcdqqach.GapsInTimeIntervalBySymbolsCheck(
        config["start_timestamp"],
        config["end_timestamp"],
        config["bid_ask_frequency_sec"],
        align=True,
    ),
]

In [11]:
dataset_validator = imvcdqdava.DataFrameDatasetValidator(qa_check_list)

In [12]:
try:
    dataset_validator.run_all_checks([data])
except Exception as e:
    # Pass information about success or failure of the QA
    #  back to the task that invoked it.
    data_qa_outcome = str(e)
    raise e
# If no exception was raised mark the QA as successful.
data_qa_outcome = "SUCCESS"

INFO  Running all QA checks:
INFO  	NaNChecks: PASSED
INFO  	FullUniversePresentCheck: PASSED
INFO  	GapsInTimeIntervalBySymbolsCheck: PASSED


In [13]:
# This can be read by the invoke task to find out if QA was successful.
hio.to_file("/app/ck_data_reconciliation_outcome.txt", data_qa_outcome)